<a href="https://colab.research.google.com/github/stas420/machine_learning_ev_ice/blob/main/EV_to_ICE_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TODO:** poprawić to, żeby robiło *git push* albo *git clone link* w zależności od potrzeby

In [ ]:
!git clone https://github.com/stas420/machine_learning_ev_ice.git
%cd machine_learning_ev_ice

In [1]:
# Setup dependencies for the project
%pip install -r requirements.txt

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Launch labelImg
!labelImg

In [3]:
import torch

rand = torch.rand(5, 3)
print(rand)

tensor([[0.5824, 0.2493, 0.6837],
        [0.4358, 0.8300, 0.4366],
        [0.1966, 0.3399, 0.1147],
        [0.5175, 0.5684, 0.9928],
        [0.9110, 0.7190, 0.0368]])


Dżem dobry
:)